In [1]:
%info

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1588603162430_0001,pyspark,idle,Link,Link,


In [3]:
sc.install_pypi_package("pandas==1.0.3")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Using cached pandas-1.0.3-cp36-cp36m-manylinux1_x86_64.whl (10.0 MB)
  Using cached python_dateutil-2.8.1-py2.py3-none-any.whl (227 kB)

In [4]:
sc.install_pypi_package("matplotlib==3.2.1")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Using cached matplotlib-3.2.1-cp36-cp36m-manylinux1_x86_64.whl (12.4 MB)
  Using cached pyparsing-2.4.7-py2.py3-none-any.whl (67 kB)
  Using cached kiwisolver-1.2.0-cp36-cp36m-manylinux1_x86_64.whl (88 kB)
  Using cached cycler-0.10.0-py2.py3-none-any.whl (6.5 kB)

In [6]:
user=spark.read.json('s3://sta9760-laura-spark-datasets/yelp-dataset/yelp_academic_dataset_business.json')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
print(f'Total Columns: {len(user.dtypes)}')
print(f'Total Rows: {user.count():,}')
user.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Total Columns: 14
Total Rows: 209,393
root
 |-- address: string (nullable = true)
 |-- attributes: struct (nullable = true)
 |    |-- AcceptsInsurance: string (nullable = true)
 |    |-- AgesAllowed: string (nullable = true)
 |    |-- Alcohol: string (nullable = true)
 |    |-- Ambience: string (nullable = true)
 |    |-- BYOB: string (nullable = true)
 |    |-- BYOBCorkage: string (nullable = true)
 |    |-- BestNights: string (nullable = true)
 |    |-- BikeParking: string (nullable = true)
 |    |-- BusinessAcceptsBitcoin: string (nullable = true)
 |    |-- BusinessAcceptsCreditCards: string (nullable = true)
 |    |-- BusinessParking: string (nullable = true)
 |    |-- ByAppointmentOnly: string (nullable = true)
 |    |-- Caters: string (nullable = true)
 |    |-- CoatCheck: string (nullable = true)
 |    |-- Corkage: string (nullable = true)
 |    |-- DietaryRestrictions: string (nullable = true)
 |    |-- DogsAllowed: string (nullable = true)
 |    |-- DriveThru: string (nullable

In [22]:
from pyspark.sql.types import DecimalType
from pyspark.sql.functions import col
#user.select('business_id','stars').withColumn('stars', col("stars").cast(DecimalType(25,16))).show(5)
user.select('business_id','stars').show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----+
|         business_id|stars|
+--------------------+-----+
|f9NumwFMBDn751xgF...|  3.5|
|Yzvjg0SayhoZgCljU...|  5.0|
|XNoUzKckATkOD1hP6...|  5.0|
|6OAZjbxqM5ol29BuH...|  2.5|
|51M2Kk903DFYI6gnB...|  4.5|
+--------------------+-----+
only showing top 5 rows

In [48]:
user_avg=user.groupby('business_id').agg({'stars':'mean'})
#user_avg=user.groupby('business_id').avg('stars')
user_avg.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----------+
|         business_id|avg(stars)|
+--------------------+----------+
|M_guz7Dj7hX0evS67...|       3.5|
|oqbNgOBAxsa49c2WQ...|       3.5|
|2BKKUjLSocdlgfsMC...|       5.0|
|DloBjCZHlz0lzsrQJ...|       4.0|
|lj0MiK5_fyv9df2tw...|       4.5|
+--------------------+----------+
only showing top 5 rows

In [24]:
business = spark.read.json('s3://sta9760-laura-spark-datasets/yelp-dataset/yelp_academic_dataset_business.json')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [43]:
user_business = user.select('business_id','stars').join(business.select('business_id','city','state'), on="business_id", how="inner")
#select("user.stars","business.name","business.city","business.state")
user_business.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----+---------------+-----+
|         business_id|stars|           city|state|
+--------------------+-----+---------------+-----+
|f9NumwFMBDn751xgF...|  3.5|      Cornelius|   NC|
|Yzvjg0SayhoZgCljU...|  5.0|     Scottsdale|   AZ|
|XNoUzKckATkOD1hP6...|  5.0|       Montreal|   QC|
|6OAZjbxqM5ol29BuH...|  2.5|North Las Vegas|   NV|
|51M2Kk903DFYI6gnB...|  4.5|           Mesa|   AZ|
+--------------------+-----+---------------+-----+
only showing top 5 rows

In [49]:
user_business_star = user_business.join(user_avg, on="business_id", how="inner")
user_business_star.show(50)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----+---------------+-----+----------+
|         business_id|stars|           city|state|avg(stars)|
+--------------------+-----+---------------+-----+----------+
|f9NumwFMBDn751xgF...|  3.5|      Cornelius|   NC|       3.5|
|Yzvjg0SayhoZgCljU...|  5.0|     Scottsdale|   AZ|       5.0|
|XNoUzKckATkOD1hP6...|  5.0|       Montreal|   QC|       5.0|
|6OAZjbxqM5ol29BuH...|  2.5|North Las Vegas|   NV|       2.5|
|51M2Kk903DFYI6gnB...|  4.5|           Mesa|   AZ|       4.5|
|cKyLV5oWZJ2NudWgq...|  4.5|        Gilbert|   AZ|       4.5|
|oiAlXZPIFm2nBCt0D...|  3.5|      Las Vegas|   NV|       3.5|
|ScYkbYNkDgCneBrD9...|  5.0|           Mesa|   AZ|       5.0|
|pQeaRpvuhoEqudo3u...|  4.5|      Champaign|   IL|       4.5|
|EosRKXIGeSWFYWwpk...|  3.0|        Toronto|   ON|       3.0|
|MbZMmwo-eL0Jnm_Yb...|  5.0|        Calgary|   AB|       5.0|
|7Dv4_HAxsxvadEsT5...|  5.0|     Pittsburgh|   PA|       5.0|
|M_guz7Dj7hX0evS67...|  3.5|      Middleton|   WI|       3.5|
|JjJs3o6